In [1]:
# ================================================
# Cell 1 NOTEBOOK 3 — LIVE TRADING ENGINE
# ================================================
import os, glob, time, json
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import gymnasium as gym
from typing import Dict, List, Optional
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize


In [2]:
# ================================================
# Cell 2 - Paths (update if needed)
# ================================================
DATA_DIR = os.path.join("data", "multiasset")
MODEL_DIR = os.path.join("models", "multiasset")
EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")
ASSET_MAP_FILE = os.path.join(DATA_DIR, "asset_to_idx.csv")
MODEL_FILE = os.path.join(MODEL_DIR, "ppo_multiasset.zip")
VEC_FILE = os.path.join(MODEL_DIR, "vec_normalize.pkl")

LOG_DIR = os.path.join(MODEL_DIR, "live_logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "live_trade_logs.csv")

In [3]:
# ================================================
# Cell 3 - Configuration
# ================================================
DRY_RUN = False            # SET True for tests. Set False only after demo tests.
WINDOW = 50 # Minimum required lookback window for analysis (e.g., for indicators)
BUFFER = 60
COUNT = WINDOW + BUFFER
DEFAULT_LOT = 1
MAX_POS_PER_SYMBOL = 1
DEFAULT_SL_PIPS_FALLBACK = 100
TP_MULT = 3
TRAIL_PIPS_DEFAULT = 10
MAGIC = 2025001          # Unique identifier for trades
COMMENT = "Sleekster_AI_Bot"
DEVIATION = 20  
# NEW: Risk Management Configuration
RISK_PERCENTAGE = 0.02    # Risk 1% (0.02) of account balance per trade
ATR_PERIOD = 14
ATR_MULTIPLIER = 2.0      # Stop Loss is set at 2.0 * ATR value

##############################
SL_OFFSET_PIPS = 10000      # Stop Loss offset in pips
TP_OFFSET_PIPS = 20000      # Take Profit offset in pips
# List to collect all trade results
ALL_TRADE_RESULTS = []


In [4]:
# ================================================
# Cell 4 - Timeframe
# ================================================
TIMEFRAME = "M15"
TF_MAP = {"M1": mt5.TIMEFRAME_M1, "M5": mt5.TIMEFRAME_M5, "M15": mt5.TIMEFRAME_M15,
          "M30": mt5.TIMEFRAME_M30, "H1": mt5.TIMEFRAME_H1, "H4": mt5.TIMEFRAME_H4, "D1": mt5.TIMEFRAME_D1}
TF_MT5 = TF_MAP.get(TIMEFRAME.upper(), mt5.TIMEFRAME_M15)


In [5]:
# ================================================
# Cell 5 - MT5 connection
# ================================================
def mt5_connect() -> bool:
    try:
        if not mt5.initialize():
            # try again to get more helpful info
            err = mt5.last_error()
            print("mt5.initialize() first attempt returned False:", err)
            ok = mt5.initialize()
            if not ok:
                print("mt5.initialize() retry failed:", mt5.last_error())
                return False
        print("MT5 connected. Version:", mt5.version())
        return True
    except Exception as e:
        print("MT5 connection error:", e)
        return False

def mt5_shutdown():
    try:
        mt5.shutdown()
    except Exception:
        pass


In [6]:
# ================================================
# Cell 6 - Symbols sellection
# ================================================
SYMBOLS: List[str] = [
    "Volatility 10 Index","Volatility 25 Index","Volatility 50 Index","Volatility 75 Index","Volatility 100 Index",
    "Volatility 10 (1s) Index","Volatility 25 (1s) Index","Volatility 50 (1s) Index","Volatility 75 (1s) Index","Volatility 100 (1s) Index",
    "Volatility 10 (10s) Index","Volatility 25 (10s) Index","Volatility 50 (10s) Index","Volatility 75 (10s) Index","Volatility 100 (10s) Index",
    "Jump 10 Index","Jump 25 Index","Jump 50 Index","Jump 75 Index","Jump 100 Index",
    "Step Index 25","Step Index 50","Step Index 75","Step Index 100","EURUSD"
]

In [6]:
# List of symbols to trade
SYMBOLS: List[str] = [
    # Volatility Indices
    "Volatility 10 Index","Volatility 25 Index","Volatility 50 Index","Volatility 75 Index","Volatility 100 Index",
    "Volatility 10 (1s) Index","Volatility 25 (1s) Index","Volatility 50 (1s) Index","Volatility 75 (1s) Index","Volatility 100 (1s) Index",
    "Volatility 10 (10s) Index","Volatility 25 (10s) Index","Volatility 50 (10s) Index","Volatility 75 (10s) Index","Volatility 100 (10s) Index",
    # Jump and Step Indices
    "Jump 10 Index","Jump 25 Index","Jump 50 Index","Jump 75 Index","Jump 100 Index",
    "Step Index 25","Step Index 50","Step Index 75","Step Index 100",
    # Major Currency Pairs Added
    "EURUSD", "GBPUSD", "USDJPY", "AUDUSD", "USDCAD", "USDCHF", "NZDUSD", "EURGBP", "EURJPY"
]

In [7]:
# ================= Full Trade Execution ================= This can trade

# If broker uses different symbol names, map safe_name -> broker_symbol
MANUAL_SYMBOL_MAP: Dict[str, str] = {
    # example: "Volatility_75_Index": "DerivVol75"
    # add entries if MT5 symbol_info(name) would be None for the plain name
}

# ----------------- Helpers -----------------
def make_safe_name(sym: str) -> str:
    return sym.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(".", "_")

def get_mt5_symbol_from_safe(safe_name: str) -> str:
    return MANUAL_SYMBOL_MAP.get(safe_name, safe_name.replace("_", " "))

In [10]:
# ================================================
# Cell 7 - Load PPO model, VecNormalize, embeddings, scalers, datasets 
# ================================================
print("Loading model and preprocessors...")
if not os.path.exists(MODEL_FILE):
    raise FileNotFoundError("Model file missing: " + MODEL_FILE)
model = PPO.load(MODEL_FILE)
print("PPO model loaded:", MODEL_FILE)

# load embeddings (attempt different formats)
embeddings = {}
EMBED_DIM = 8
if os.path.exists(EMBED_FILE):
    try:
        emb_loaded = np.load(EMBED_FILE, allow_pickle=True)
        # if saved dict
        if isinstance(emb_loaded, np.ndarray) and emb_loaded.dtype == object:
            try:
                obj = emb_loaded.item()
                if isinstance(obj, dict):
                    embeddings = {k: np.array(v, dtype=np.float32) for k, v in obj.items()}
            except Exception:
                pass
        if len(embeddings) == 0:
            # if array mapping by index
            try:
                arr = np.array(emb_loaded)
                if arr.ndim == 2:
                    # attempt mapping with asset_to_idx
                    if os.path.exists(ASSET_MAP_FILE):
                        try:
                            am = pd.read_csv(ASSET_MAP_FILE, index_col=0, header=None).iloc[:,0].to_dict()
                            for safe, idx in am.items():
                                idx = int(idx)
                                if idx < arr.shape[0]:
                                    embeddings[safe] = np.array(arr[idx], dtype=np.float32)
                        except Exception:
                            pass
                # final fallback: if embeddings length matches input CSV count, map by CSV order in data dir
                if len(embeddings) == 0 and arr.ndim == 2:
                    csvs = sorted(glob.glob(os.path.join(DATA_DIR, "*_normalized.csv")))
                    safe_list = [os.path.basename(p).replace("_normalized.csv","") for p in csvs]
                    if len(safe_list) == arr.shape[0]:
                        for i, s in enumerate(safe_list):
                            embeddings[s] = np.array(arr[i], dtype=np.float32)
            except Exception:
                pass
    except Exception as e:
        print("Failed to load embeddings:", e)
if len(embeddings) > 0:
    EMBED_DIM = next(iter(embeddings.values())).shape[0]
print("Embeddings loaded:", len(embeddings), "embed_dim:", EMBED_DIM)


# VecNormalize (safe load using dummy env)
vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        class _DummyEnv(gym.Env):
            def __init__(self):
                super().__init__()
                self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(WINDOW, 5 + 1 + EMBED_DIM), dtype=np.float32)
                self.action_space = gym.spaces.Discrete(3)
            def reset(self, seed=None, options=None):
                return np.zeros(self.observation_space.shape, dtype=np.float32), {}
            def step(self, action):
                return np.zeros(self.observation_space.shape, dtype=np.float32), 0.0, True, False, {}
        venv = DummyVecEnv([lambda: _DummyEnv()])
        vecnorm = VecNormalize.load(VEC_FILE, venv)
        vecnorm.training = False
        vecnorm.norm_reward = False
        print("VecNormalize loaded:", VEC_FILE)
    except Exception as e:
        print("VecNormalize load failed (continuing without it):", e)
        vecnorm = None
else:
    print("No VecNormalize file found; continuing without it.")


Loading model and preprocessors...
PPO model loaded: models\multiasset\ppo_multiasset.zip
Embeddings loaded: 17 embed_dim: 8
VecNormalize loaded: models\multiasset\vec_normalize.pkl


In [11]:
# ================================================
# Cell 7 - LOAD SCALERS 
# ================================================
# scalers
def load_scalers(data_dir=DATA_DIR):
    scalers = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_scaler.csv"))):
        safe = os.path.basename(p).replace("_scaler.csv","")
        df = pd.read_csv(p, index_col=0)
        scalers[safe] = {"mean": df["mean"], "std": df["std"]}
    return scalers

scalers = load_scalers()
print("Scalers loaded:", len(scalers))

Scalers loaded: 16


In [12]:
# ================================================
# Cell 8 - datasets (prepared normalized CSVs)
# ================================================
def load_prepared_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            tmp = df[expected].dropna()
            if len(tmp) > window:
                datasets[safe] = tmp
        else:
            # try convert from OHLCV if possible
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                tmp = tmp.dropna()
                if len(tmp) > window:
                    datasets[safe] = tmp
    return datasets

datasets = load_prepared_datasets()
print("Prepared datasets loaded:", len(datasets))

Prepared datasets loaded: 16


In [13]:
# ================================================
# Cell 9 - Live bars fetch
# ================================================
def _fetch_single_symbol_bars(mt5_symbol: str, timeframe: int, count: int) -> Optional[pd.DataFrame]:
    info = mt5.symbol_info(mt5_symbol)
    if info is None:
        # no such symbol in terminal
        return None
    if not info.visible:
        try:
            mt5.symbol_select(mt5_symbol, True)
        except Exception:
            pass
    end_time = datetime.now()
    bars = mt5.copy_rates_from(mt5_symbol, timeframe, end_time, count)
    if bars is None or len(bars) < WINDOW + 2:
        return None
    df = pd.DataFrame(bars)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df.set_index('time')
    if 'tick_volume' in df.columns:
        df = df.rename(columns={'tick_volume':'volume'})
    df = df[['open','high','low','close','volume']].copy()
    df['Close_raw'] = df['close']
    return df

In [14]:
# ----------------- Observation builder -----------------
def build_obs_for_symbol(mt5_symbol: str, safe_name: str, window: int = WINDOW):
    # try live bars
    df_live = _fetch_single_symbol_bars(mt5_symbol, TF_MT5, COUNT)
    if df_live is None:
        # fallback to prepared dataset
        if safe_name not in datasets:
            return None, None, None
        df_pct = datasets[safe_name].iloc[-window:][['o_pc','h_pc','l_pc','c_pc','v_pc']]
        last_price = float(datasets[safe_name]['Close_raw'].iloc[-1])
        features = df_pct.values.astype(np.float32)
    else:
        # compute percent changes from live OHLCV
        df_pct = df_live[['open','high','low','close','volume']].pct_change().dropna()
        if len(df_pct) < window:
            return None, None, None
        df_pct = df_pct.tail(window)
        last_price = float(df_live['Close_raw'].iloc[-1])
        features = df_pct[['open','high','low','close','volume']].values.astype(np.float32)

    # scaler required
    if safe_name not in scalers:
        return None, None, None
    s = scalers[safe_name]
    mean = np.array(s['mean'].values if hasattr(s['mean'],'values') else s['mean'], dtype=np.float32)
    std = np.array(s['std'].values if hasattr(s['std'],'values') else s['std'], dtype=np.float32)
    std = np.where(std == 0, 1e-8, std)

    if features.shape[1] != len(mean):
        return None, None, None

    features_scaled = (features - mean) / std

    # embedding
    emb_vec = embeddings.get(safe_name, np.zeros(EMBED_DIM, dtype=np.float32))
    emb_block = np.repeat(emb_vec.reshape(1,-1), window, axis=0)

    # balance norm column (training used some balance column — here we set 1.0)
    balance_col = np.ones((window,1), dtype=np.float32)

    obs = np.concatenate([features_scaled.astype(np.float32), balance_col, emb_block], axis=1)

    # apply VecNormalize if available (best-effort)
    if vecnorm is not None:
        try:
            # prefer normalize_obs if available
            try:
                obs = vecnorm.normalize_obs(obs, False)
            except Exception:
                # fallback to obs_rms stats
                flat = obs.reshape(1,-1)
                mean_rms = getattr(vecnorm.obs_rms, "mean", None)
                var_rms = getattr(vecnorm.obs_rms, "var", None)
                if mean_rms is not None and var_rms is not None and len(mean_rms) == flat.shape[1]:
                    flat_norm = (flat - mean_rms) / np.sqrt(var_rms + 1e-8)
                    obs = flat_norm.reshape(obs.shape).astype(np.float32)
        except Exception:
            pass

    vol_est = float(np.std(features[:,3]))  # std of close pct
    return obs.astype(np.float32), vol_est, last_price


In [15]:
# Calculate Optimum Lot-size
def calculate_optimal_volume(symbol: str, entry_price: float, sl_price: float, risk_percent: float) -> float:
    """
    Calculates the optimal lot size based on account equity, risk percentage,
    and the stop loss distance, constrained by symbol limits.

    The formula is: OptimalLotSize = (Equity * RiskPercentage) / (abs(EntryPrice - SLPrice) * ContractSize)
    """
    
    # 1. Get Account Equity
    account_info = mt5.account_info()
    if account_info is None:
        print("Error: Could not retrieve account information.")
        return 0.0
    acc_equity = account_info.balance
    
    # 2. Get Symbol Information for contract size and volume limits
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Error: Could not retrieve symbol info for {symbol}.")
        return 0.0
    
    contract_size = symbol_info.trade_contract_size
    min_volume = symbol_info.volume_min
    max_volume = symbol_info.volume_max
    volume_step = symbol_info.volume_step
    
    # Check for zero values to prevent division by zero
    if contract_size == 0:
        print(f"Error: Contract size is zero for {symbol}. Returning minimum volume.")
        return min_volume
        
    # 3. Calculate Maximum Dollar Loss
    max_loss_dollar = acc_equity * risk_percent
    
    # 4. Calculate Loss per Lot (Loss in price difference * Contract Size)
    price_loss = abs(entry_price - sl_price)
    
    # If SL is too close or equal to entry, use min volume as a fallback.
    if price_loss == 0.0:
        print(f"Warning: SL is equal to entry price for {symbol}. Using minimum volume.")
        return min_volume

    loss_per_lot = price_loss * contract_size
    
    # 5. Calculate Optimal Lot Size (Theoretical)
    optimal_lot_size = max_loss_dollar / loss_per_lot
    
    # 6. Apply Volume Step and Min/Max constraints
    
    # Snap the calculated lot size to the nearest allowed volume step (e.g., 0.01)
    optimal_lot_size = round(optimal_lot_size / volume_step) * volume_step
    
    # Apply min/max volume limits
    final_volume = max(min_volume, min(max_volume, optimal_lot_size))
        
    print(f"  [Risk Calcs] Equity: {acc_equity:.2f}, Risk %: {risk_percent*100}%, Max Loss $: {max_loss_dollar:.2f}, Final Volume: {final_volume:.2f}")
    
    # Ensure the final volume is rounded to the step size's precision
    final_volume = round(final_volume, len(str(volume_step).split('.')[-1]) if '.' in str(volume_step) else 0)

    return final_volume


In [16]:
def get_pip_value(symbol: str) -> float | None:
    """
    Retrieves the point value (often used as the pip value) for a given symbol.
    
    This function is self-contained and useful for calculating stop/take profit
    distances outside of the main trade execution flow.
    
    :param symbol: The trading instrument name.
    :return: The point value (float) or None if the symbol is not found.
    """
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Error: Symbol {symbol} not found.")
        return None
    
    # In MT5 programming, the symbol's 'point' is the smallest change unit
    # and is typically used for pips-based distance calculations.
    return symbol_info.point

In [45]:
def mt5_ensure_init():
    try:
        return mt5.initialize()
    except Exception:
        return False

def get_symbol_info(symbol):
    if not mt5_ensure_init():
        return None
    try:
        return mt5.symbol_info(symbol)
    except Exception:
        return None

def pip_value_per_lot_from_mt5(symbol, entry_price):
    info = get_symbol_info(symbol)
    if info is None:
        return None, None
    try:
        pip_val = info.trade_tick_value / info.trade_tick_size
        print("trade_tick_value", info.trade_tick_value)
        print("trade_tick_size", info.trade_tick_size)
        print("point", info.point)
        print("contract_size", info.contract_size)
    except Exception:
        contract_size = getattr(info, "trade_contract_size", 100000.0)
        pip_val = (info.point / entry_price) * contract_size
    return float(pip_val), float(info.point)

In [46]:
PIPVALUE = pip_value_per_lot_from_mt5("EURUSD", 1.16680)

trade_tick_value 1.0
trade_tick_size 1e-05
point 1e-05


In [44]:
PIPVALUE

(0.8570449091532397, 1e-05)

In [33]:
# Main functions
# ----------------- Trading helpers -----------------
def estimate_sl_pips(symbol: str, last_price: float, vol_est: float, min_pips=50, max_pips=500) -> int:
    pip = get_pip_value(symbol)
    abs_move = vol_est * last_price
    if abs_move <= 0:
        return DEFAULT_SL_PIPS_FALLBACK
    sl_raw = abs_move / pip
    sl_pips = int(np.clip(np.round(sl_raw * 2.5), min_pips, max_pips))
    return max(1, sl_pips)

def compute_sl_tp_prices(symbol: str, price: float, direction: str, sl_pips: int, tp_pips: int):
    pip = pip_value(symbol)
    if direction == "BUY":
        sl = price - sl_pips * pip
        tp = price + tp_pips * pip
    else:
        sl = price + sl_pips * pip
        tp = price - tp_pips * pip
    return float(sl), float(tp)

def get_positions_for_symbol(symbol: str):
    pos = mt5.positions_get(symbol=symbol)
    return [] if pos is None else list(pos)


In [16]:
def calculate_atr(symbol: str, timeframe: int, period: int) -> float:
    """
    Calculates the Simple Average True Range (SATR) over the specified period.
    MQL5 logic: ATR = AvgTrueRangeSum = Sum(TR) / period
    
    :param symbol: Trading instrument
    :param timeframe: Timeframe for historical data (e.g., mt5.TIMEFRAME_M1)
    :param period: Number of bars (N) for the calculation
    :return: ATR value (price movement)
    """
    # Request N+1 bars to ensure we have the necessary previous close price (C_n-1)
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 1, period + 1)
    
    if rates is None or len(rates) < period + 1:
        print(f"  [ATR Error] Failed to get enough historical data for {symbol}.")
        return 0.0

    rates_df = pd.DataFrame(rates)
    
    # Calculate True Range (TR) for N bars (indices 0 to N-1)
    # 1. High - Low
    range1 = rates_df['high'] - rates_df['low']
    
    # 2. Abs(High - Previous Close)
    # Note: Shift(1) moves prices down, so rates_df['close'].shift(1) gives the *next older* bar's close.
    # We want the close of the bar *immediately following* the current one in the array (older in time).
    # Since we fetched bars from the past: rates_df[0] is newest, rates_df[period] is oldest.
    # We need the previous bar's close (C_n-1) which is at index i+1 for bar i.
    previous_close = rates_df['close'].shift(-1)
    range2 = abs(rates_df['high'] - previous_close)
    
    # 3. Abs(Low - Previous Close)
    range3 = abs(rates_df['low'] - previous_close)
    
    # Combine ranges (max of the three for each bar)
    # We only need TR for the most recent 'period' bars (indices 0 to period-1)
    tr = pd.concat([range1, range2, range3], axis=1).max(axis=1)
    
    # Calculate the Simple Average True Range (SATR) over the 'period' bars (excluding the last row/oldest bar which has NaN TR)
    atr_value = tr.iloc[0:period].mean()

    # If the calculation fails (e.g., first few bars of history), return 0.0
    if pd.isna(atr_value):
        print("  [ATR Error] ATR calculation resulted in NaN.")
        return 0.0
        
    print(f"  [ATR Calcs] ATR({period}) value: {atr_value:.8f}")
    
    return atr_value * ATR_MULTIPLIER



# Calculate ATR value (price distance)
atr_price_distance = calculate_atr(symbol, TF_MT5, ATR_PERIOD)
    
    # Use ATR * Multiplier for the SL distance. If ATR fails, use a fallback distance based on TP PIPS offset.
#if atr_price_distance > 0.0:
#    sl_distance = atr_price_distance * ATR_MULTIPLIER
#    print(f"  [SL Setup] Using ATR-based SL distance: {sl_distance:.{symbol_info.digits}f}")
#else:
#    # Fallback to the fixed PIPS offset, converted to a price value
#    sl_distance = TP_OFFSET_PIPS * pip_value
#    print(f"  [SL Setup] ATR failed. Using fixed SL distance: {sl_distance:.{symbol_info.digits}f}")

In [26]:
def place_market_order(symbol: str, direction: str, sl_price: float, tp_price: float):

    # Get current price
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Failed to get current tick for {symbol}.")
        # Return a dictionary indicating local failure before calling format_result
        return None, {"symbol": symbol, "type": direction, "volume": 0.0} 

    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return {"retcode": None, "comment": "NO_TICK"}
    price = float(tick.ask if direction == "BUY" else tick.bid)
    if DRY_RUN:
        return {"retcode": 10009, "comment": "DRY_RUN", "price": price}

    lot = calculate_optimal_volume(symbol, price, sl_price, RISK_PERCENTAGE)
    
    if lot <= 0.0005:
        print(f"Skipping trade for {broker_sym}: Calculated volume is zero or negative.")
        # Return the request dictionary with 0 volume to be logged as a skipped trade
        return None, {"symbol": broker_sym, "type": direction, "volume": 0.0}
    
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(lot),
        "type": mt5.ORDER_TYPE_BUY if direction=="BUY" else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": float(sl_price) if sl_price is not None else 0.0,
        "tp": float(tp_price) if tp_price is not None else 0.0,
        #"sl": round(sl_price, symbol_info.digits), 
        #"tp": round(tp_price, symbol_info.digits),
        "deviation": DEVIATION,
        "magic": MAGIC,
        "comment": "ppo_multiasset_live",
        "type_filling": mt5.ORDER_FILLING_FOK,
    }
    
    return mt5.order_send(req)

In [19]:
# ================================================
# Cell 1 SEND LIVE MARKET ODER -BEST-
# ================================================
def place_market_order(symbol_info, order_type: int, lot: float, stopLoss:float, takeProfit:float): 
    """
    Constructs and sends a market order (BUY or SELL) using a dynamically calculated volume.
    
    :param symbol_info: mt5.symbol_info object
    :param pip_value: The size of a single pip (point) for the symbol
    :param order_type: mt5.ORDER_TYPE_BUY or mt5.ORDER_TYPE_SELL
    """
    pip_value = 0.3
    symbol = symbol_info.name
    
    # Get current price
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Failed to get current tick for {symbol}.")
        # Return a dictionary indicating local failure before calling format_result
        #return None, {"symbol": symbol, "type": order_type, "volume": 0.0} 

    # Determine execution price, trade type, and SL/TP prices
    if order_type == mt5.ORDER_TYPE_BUY:
        price = tick.ask
        type_text = "BUY"
        # SL/TP calculation (SL below, TP above entry price for BUY)
        sl_price = price - (SL_OFFSET_PIPS * pip_value)
        tp_price = price + (TP_OFFSET_PIPS * pip_value)
    elif order_type == mt5.ORDER_TYPE_SELL:
        price = tick.bid
        type_text = "SELL"
        # SL/TP calculation (SL above, TP below entry price for SELL)
        sl_price = price + (SL_OFFSET_PIPS * pip_value)
        tp_price = price - (TP_OFFSET_PIPS * pip_value)
    else:
        print("Invalid order type.")
        return None, None
        
    # --- DYNAMIC VOLUME CALCULATION ---
    volume_to_use = calculate_optimal_volume(
        symbol, 
        price, 
        sl_price, 
        RISK_PERCENTAGE
    )
    
    if volume_to_use <= 0.0:
        print(f"Skipping trade for {symbol}: Calculated volume is zero or negative.")
        # Return the request dictionary with 0 volume to be logged as a skipped trade
        return None, {"symbol": symbol, "type": order_type, "volume": 0.0}

    # Prepare the request dictionary
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume_to_use, # <-- Dynamic volume used here
        "type": order_type,
        "price": price,
        # Round SL/TP based on the symbol's number of digits
        "sl": round(sl_price, symbol_info.digits), 
        "tp": round(tp_price, symbol_info.digits), 
        "deviation": DEVIATION,
        "magic": MAGIC,
        "comment": COMMENT,
        "type_time": mt5.ORDER_TIME_GTC, 
        "type_filling": mt5.ORDER_FILLING_FOK, 
    }

    # Send order to MT5
    print(f"  Sending {type_text} order for {symbol} with {volume_to_use:.2f} lots at {price:.{symbol_info.digits}f}...")
    result = mt5.order_send(request)

    return result, request



In [18]:
def close_position_by_ticket(ticket: int):
    pos_list = mt5.positions_get(ticket=ticket)
    if not pos_list:
        return None
    p = pos_list[0]
    symbol = p.symbol
    if getattr(p,"type",None) == 0:  # BUY
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    else:
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    if DRY_RUN:
        return {"retcode":10009, "comment":"DRY_RUN_CLOSE", "ticket": ticket}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(p.volume),
        "type": order_type,
        "position": int(ticket),
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": "auto_close",
    }
    return mt5.order_send(req)


In [34]:
# ----------------- Main run loop: single pass -----------------
def run_once_predict_and_manage(symbols_list: List[str]):
    if not mt5_connect():
        print("MT5 connect failed — aborting.")
        return

    acct = mt5.account_info()
    balance = float(acct.balance) if acct is not None else 10000.0
    header = not os.path.exists(LOG_FILE)

    for raw_sym in symbols_list:
        safe_name = make_safe_name(raw_sym)
        broker_sym = get_mt5_symbol_from_safe(safe_name)
        print(f"\n=== {broker_sym} (safe: {safe_name}) ===")

        obs_tuple = build_obs_for_symbol(broker_sym, safe_name, WINDOW)
        if obs_tuple is None:
            print("No observation for", safe_name, "- skipping")
            continue
        obs, vol, last_price = obs_tuple

        # predict
        try:
            action, _ = model.predict(obs[np.newaxis, ...], deterministic=True)
            a = int(action[0]) if isinstance(action, (list, tuple, np.ndarray)) else int(action)
        except Exception as e:
            print("Prediction error:", e)
            continue

        print("Signal:", a, "(0=HOLD,1=BUY,2=SELL)")

        # manage positions
        positions = get_positions_for_symbol(broker_sym)
        print("Existing positions:", len(positions))

        # auto-close opposing positions
        if a == 1:  # BUY signal -> close SELL positions
            for p in positions:
                if getattr(p, "type", None) == 1:
                    print("Closing opposing SELL ticket", p.ticket)
                    close_position_by_ticket(p.ticket)
        elif a == 2:  # SELL signal -> close BUY positions
            for p in positions:
                if getattr(p, "type", None) == 0:
                    print("Closing opposing BUY ticket", p.ticket)
                    close_position_by_ticket(p.ticket)

        # refresh
        positions = get_positions_for_symbol(broker_sym)
        if len(positions) >= MAX_POS_PER_SYMBOL:
            print("Max positions reached for", broker_sym, "- skipping open")
        else:
            if a == 0:
                print("HOLD")
            else:
                direction = "BUY" if a == 1 else "SELL"
                sl_pips = estimate_sl_pips(broker_sym, last_price, vol)
                tp_pips = int(sl_pips * TP_MULT)
                sl_price, tp_price = compute_sl_tp_prices(broker_sym, last_price, direction, sl_pips, tp_pips)
                #res = place_market_order(broker_sym, direction, 2, sl_price, tp_price)
                #res = place_market_order(broker_sym, direction, sl_price, tp_price)
                SL = calculate_atr(broker_sym, TF_MT5, ATR_PERIOD)
                # --- DYNAMIC VOLUME CALCULATION ---
                #lot = calculate_optimal_volume(broker_sym, price, sl_price, RISK_PERCENTAGE)
                    
                #res = place_market_order(broker_sym, direction, 0, 0)
                res = place_market_order(broker_sym, direction, sl_price, 0)
                if isinstance(res, dict):
                    retcode = res.get("retcode")
                    comment = res.get("comment")
                    exec_price = res.get("price", last_price)
                else:
                    retcode = getattr(res, "retcode", None)
                    comment = getattr(res, "comment", "")
                    exec_price = last_price

                entry = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "safe": safe_name,
                    "symbol": broker_sym,
                    "action": direction,
                    "lot": 1,
                    "exec_price": exec_price,
                    "sl_price": sl_price,
                    "tp_price": tp_price,
                    "sl_pips": sl_pips,
                    "tp_pips": tp_pips,
                    "retcode": retcode,
                    "comment": comment,
                    "dry_run": DRY_RUN
                }
                pd.DataFrame([entry]).to_csv(LOG_FILE, mode="a", index=False, header=header)
                header = False
                #print("Placed", direction, "lot", lot, "retcode", retcode)
                print("Placed", direction, "lot", 1, "retcode", retcode)

        # trailing SL pass for current positions
        for p in get_positions_for_symbol(broker_sym):
            trail_pips = max(5, int(estimate_sl_pips(broker_sym, last_price, vol) // 2))
            pip = pip_value(broker_sym)
            if getattr(p, "type", None) == 0:  # BUY
                cur_price = mt5.symbol_info_tick(broker_sym).bid
                new_sl = float(cur_price - trail_pips * pip)
            else:
                cur_price = mt5.symbol_info_tick(broker_sym).ask
                new_sl = float(cur_price + trail_pips * pip)

            if DRY_RUN:
                print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
            else:
                try:
                    req = {"action": mt5.TRADE_ACTION_SLTP, "symbol": broker_sym, "position": int(p.ticket), "sl": new_sl, "tp": float(p.tp) if getattr(p,"tp",None) else 0.0}
                    r = mt5.order_send(req)
                    print("Modify SL retcode:", getattr(r, "retcode", None))
                except Exception as e:
                    print("Modify SL failed:", e)

    mt5_shutdown()
    print("\nSingle pass complete. Logs written to:", LOG_FILE)

# ------------- run once (example) -------------
# For safety, you can test a subset:
test_symbols = SYMBOLS[:]  # change as needed
run_once_predict_and_manage(test_symbols)

# ------------- Optional continuous loop -------------
# Uncomment and use with caution (ensure DRY_RUN=False only after demo verification)
try:
    while True:
        run_once_predict_and_manage(SYMBOLS)
        time.sleep(60)   # run each minute for M1/M15 choose appropriate sleep
except KeyboardInterrupt:
    print("Stopped by user.")


MT5 connected. Version: (500, 5430, '14 Nov 2025')

=== Volatility 10 Index (safe: Volatility_10_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
  [ATR Calcs] ATR(14) value: 4.60907143
  [Risk Calcs] Equity: 6646.16, Risk %: 2.0%, Max Loss $: 132.92, Final Volume: 400.00
Placed BUY lot 1 retcode 10016

=== Volatility 25 Index (safe: Volatility_25_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
  [ATR Calcs] ATR(14) value: 6.40707143
  [Risk Calcs] Equity: 6646.16, Risk %: 2.0%, Max Loss $: 132.92, Final Volume: 400.00
Placed SELL lot 1 retcode 10016

=== Volatility 50 Index (safe: Volatility_50_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
  [ATR Calcs] ATR(14) value: 0.77221429
  [Risk Calcs] Equity: 6646.16, Risk %: 2.0%, Max Loss $: 132.92, Final Volume: 1926.42
Placed BUY lot 1 retcode 10016

=== Volatility 75 Index (safe: Volatility_75_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
  [ATR Calcs] ATR(14) value: 

C:\Users\Slick\AppData\Local\Temp\ipykernel_12404\652735078.py:78: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12404\652735078.py:78: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12404\652735078.py:78: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12404\65

TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'

In [26]:
import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime
from typing import List
import numpy as np

# --- Configuration ---
# List of symbols to trade
SYMBOL_LIST: List[str] = [
    # Volatility Indices
    "Volatility 10 Index","Volatility 25 Index","Volatility 50 Index","Volatility 75 Index","Volatility 100 Index",
    "Volatility 10 (1s) Index","Volatility 25 (1s) Index","Volatility 50 (1s) Index","Volatility 75 (1s) Index","Volatility 100 (1s) Index",
    "Volatility 10 (10s) Index","Volatility 25 (10s) Index","Volatility 50 (10s) Index","Volatility 75 (10s) Index","Volatility 100 (10s) Index",
    # Jump and Step Indices
    "Jump 10 Index","Jump 25 Index","Jump 50 Index","Jump 75 Index","Jump 100 Index",
    "Step Index 25","Step Index 50","Step Index 75","Step Index 100",
    # Major Currency Pairs Added
    "EURUSD", "GBPUSD", "USDJPY", "AUDUSD", "USDCAD", "USDCHF", "NZDUSD", "EURGBP", "EURJPY"
]
DEVIATION = 10            # Maximum acceptable price deviation
MAGIC = 20250615          # Unique identifier for trades
COMMENT = "Multi_Symbol_Trade_Bot"
# SL_OFFSET_PIPS has been replaced by ATR-based settings:
TP_OFFSET_PIPS = 200      # Fixed Take Profit offset in pips (used if ATR is zero/fails)

# NEW: Risk Management & Volatility Configuration
RISK_PERCENTAGE = 0.01    # Risk 1% (0.01) of account equity per trade
ATR_PERIOD = 14           # Number of bars for ATR calculation (e.g., 14)
ATR_MULTIPLIER = 2.0      # Stop Loss is set at 2.0 * ATR value

# List to collect all trade results
ALL_TRADE_RESULTS = []

def connect_mt5(login=None, password=None, server=None) -> bool:
    """Initializes and connects to the MT5 terminal."""
    if not mt5.initialize():
        print(f"MT5 initialization failed. Error code: {mt5.last_error()}")
        return False

    if login and password and server:
        # Try to log in if credentials are provided
        authorized = mt5.login(login, password=password, server=server)
        if not authorized:
            print(f"Login failed: {mt5.last_error()}")
            mt5.shutdown()
            return False
            
    print(f"Successfully connected to MT5 terminal.")
    account_info = mt5.account_info()
    if account_info:
        print(f"Account: {account_info.login}")
    return True

def get_symbol_info(symbol: str):
    """
    Retrieves and prepares the symbol for trading.
    Returns the symbol information object and the price tick value.
    """
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"--- {symbol} NOT FOUND ---")
        return None, None

    if not symbol_info.visible:
        # Check if the symbol can be added to Market Watch
        if not mt5.symbol_select(symbol, True):
            print(f"--- {symbol} NOT TRADABLE: Not visible and could not be added ---")
            return None, None

    # Get the tick size (point value) for SL/TP calculations
    pip_value = symbol_info.point
    
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"--- {symbol} NOT TRADABLE: Could not get tick info ---")
        return None, None
        
    print(f"\n--- Symbol Ready: {symbol} ---")
    print(f"Digits: {symbol_info.digits} | Point: {symbol_info.point} | Bid: {tick.bid} | Ask: {tick.ask}")
    
    return symbol_info, pip_value


def calculate_atr(symbol: str, timeframe: int, period: int) -> float:
    """
    Calculates the Simple Average True Range (SATR) over the specified period.
    MQL5 logic: ATR = AvgTrueRangeSum = Sum(TR) / period
    
    :param symbol: Trading instrument
    :param timeframe: Timeframe for historical data (e.g., mt5.TIMEFRAME_M1)
    :param period: Number of bars (N) for the calculation
    :return: ATR value (price movement)
    """
    # Request N+1 bars to ensure we have the necessary previous close price (C_n-1)
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 1, period + 1)
    
    if rates is None or len(rates) < period + 1:
        print(f"  [ATR Error] Failed to get enough historical data for {symbol}.")
        return 0.0

    rates_df = pd.DataFrame(rates)
    
    # Calculate True Range (TR) for N bars (indices 0 to N-1)
    # 1. High - Low
    range1 = rates_df['high'] - rates_df['low']
    
    # 2. Abs(High - Previous Close)
    # Note: Shift(1) moves prices down, so rates_df['close'].shift(1) gives the *next older* bar's close.
    # We want the close of the bar *immediately following* the current one in the array (older in time).
    # Since we fetched bars from the past: rates_df[0] is newest, rates_df[period] is oldest.
    # We need the previous bar's close (C_n-1) which is at index i+1 for bar i.
    previous_close = rates_df['close'].shift(-1)
    range2 = abs(rates_df['high'] - previous_close)
    
    # 3. Abs(Low - Previous Close)
    range3 = abs(rates_df['low'] - previous_close)
    
    # Combine ranges (max of the three for each bar)
    # We only need TR for the most recent 'period' bars (indices 0 to period-1)
    tr = pd.concat([range1, range2, range3], axis=1).max(axis=1)
    
    # Calculate the Simple Average True Range (SATR) over the 'period' bars (excluding the last row/oldest bar which has NaN TR)
    atr_value = tr.iloc[0:period].mean()

    # If the calculation fails (e.g., first few bars of history), return 0.0
    if pd.isna(atr_value):
        print("  [ATR Error] ATR calculation resulted in NaN.")
        return 0.0
        
    print(f"  [ATR Calcs] ATR({period}) value: {atr_value:.8f}")
    
    return atr_value


def calculate_optimal_volume(symbol: str, entry_price: float, sl_price: float, risk_percent: float) -> float:
    """
    Calculates the optimal lot size based on account equity, risk percentage,
    and the stop loss distance, constrained by symbol limits.

    The formula is: OptimalLotSize = (Equity * RiskPercentage) / (abs(EntryPrice - SLPrice) * ContractSize)
    """
    
    # 1. Get Account Equity
    account_info = mt5.account_info()
    if account_info is None:
        print("Error: Could not retrieve account information.")
        return 0.0
    acc_equity = account_info.equity
    
    # 2. Get Symbol Information for contract size and volume limits
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Error: Could not retrieve symbol info for {symbol}.")
        return 0.0
    
    contract_size = symbol_info.trade_contract_size
    min_volume = symbol_info.volume_min
    max_volume = symbol_info.volume_max
    volume_step = symbol_info.volume_step
    
    # Check for zero values to prevent division by zero
    if contract_size == 0:
        print(f"Error: Contract size is zero for {symbol}. Returning minimum volume.")
        return min_volume
        
    # 3. Calculate Maximum Dollar Loss
    max_loss_dollar = acc_equity * risk_percent
    
    # 4. Calculate Loss per Lot (Loss in price difference * Contract Size)
    price_loss = abs(entry_price - sl_price)
    
    # If SL is too close or equal to entry, use min volume as a fallback.
    if price_loss == 0.0:
        print(f"Warning: SL is equal to entry price for {symbol}. Using minimum volume.")
        return min_volume

    loss_per_lot = price_loss * contract_size
    
    # 5. Calculate Optimal Lot Size (Theoretical)
    optimal_lot_size = max_loss_dollar / loss_per_lot
    
    # 6. Apply Volume Step and Min/Max constraints
    
    # Snap the calculated lot size to the nearest allowed volume step (e.g., 0.01)
    optimal_lot_size = round(optimal_lot_size / volume_step) * volume_step
    
    # Apply min/max volume limits
    final_volume = max(min_volume, min(max_volume, optimal_lot_size))
        
    print(f"  [Risk Calcs] Equity: {acc_equity:.2f}, Risk %: {risk_percent*100}%, Max Loss $: {max_loss_dollar:.2f}, Final Volume: {final_volume:.2f}")
    
    # Ensure the final volume is rounded to the step size's precision
    final_volume = round(final_volume, len(str(volume_step).split('.')[-1]) if '.' in str(volume_step) else 0)

    return final_volume


def place_market_order(symbol_info, pip_value: float, order_type: int):
    """
    Constructs and sends a market order (BUY or SELL) using a dynamically calculated volume and ATR-based SL.
    
    :param symbol_info: mt5.symbol_info object
    :param pip_value: The size of a single pip (point) for the symbol
    :param order_type: mt5.ORDER_TYPE_BUY or mt5.ORDER_TYPE_SELL
    """
    symbol = symbol_info.name
    
    # Get current price
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Failed to get current tick for {symbol}.")
        # Return a dictionary indicating local failure before calling format_result
        return None, {"symbol": symbol, "type": order_type, "volume": 0.0} 

    # --- 1. DETERMINE ENTRY, SL, AND TP PRICES ---
    
    # Calculate ATR value (price distance)
    atr_price_distance = calculate_atr(symbol, mt5.TIMEFRAME_M1, ATR_PERIOD)
    
    # Use ATR * Multiplier for the SL distance. If ATR fails, use a fallback distance based on TP PIPS offset.
    if atr_price_distance > 0.0:
        sl_distance = atr_price_distance * ATR_MULTIPLIER
        print(f"  [SL Setup] Using ATR-based SL distance: {sl_distance:.{symbol_info.digits}f}")
    else:
        # Fallback to the fixed PIPS offset, converted to a price value
        sl_distance = TP_OFFSET_PIPS * pip_value
        print(f"  [SL Setup] ATR failed. Using fixed SL distance: {sl_distance:.{symbol_info.digits}f}")


    if order_type == mt5.ORDER_TYPE_BUY:
        price = tick.ask
        type_text = "BUY"
        # SL below entry price, TP above entry price
        sl_price = price - sl_distance
        tp_price = price + (TP_OFFSET_PIPS * pip_value) # Keep TP fixed for simplicity
    elif order_type == mt5.ORDER_TYPE_SELL:
        price = tick.bid
        type_text = "SELL"
        # SL above entry price, TP below entry price
        sl_price = price + sl_distance
        tp_price = price - (TP_OFFSET_PIPS * pip_value) # Keep TP fixed for simplicity
    else:
        print("Invalid order type.")
        return None, None
        
    # --- 2. DYNAMIC VOLUME CALCULATION ---
    volume_to_use = calculate_optimal_volume(
        symbol, 
        price, 
        sl_price, 
        RISK_PERCENTAGE
    )
    
    if volume_to_use <= 0.0:
        print(f"Skipping trade for {symbol}: Calculated volume is zero or negative.")
        # Return the request dictionary with 0 volume to be logged as a skipped trade
        return None, {"symbol": symbol, "type": order_type, "volume": 0.0}

    # --- 3. PREPARE AND SEND REQUEST ---
    # Prepare the request dictionary
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume_to_use, # <-- Dynamic volume used here
        "type": order_type,
        "price": price,
        # Round SL/TP based on the symbol's number of digits
        "sl": round(sl_price, symbol_info.digits), 
        "tp": round(tp_price, symbol_info.digits), 
        "deviation": DEVIATION,
        "magic": MAGIC,
        "comment": COMMENT,
        "type_time": mt5.ORDER_TIME_GTC, 
        "type_filling": mt5.ORDER_FILLING_FOK, 
    }

    # Send order to MT5
    print(f"  Sending {type_text} order for {symbol} with {volume_to_use:.2f} lots at {price:.{symbol_info.digits}f}...")
    result = mt5.order_send(request)

    return result, request


def format_result(result, request):
    """
    Formats the order result and request into a clean dictionary.
    Handles both successful and failed execution attempts.
    """
    symbol = request.get("symbol", "N/A")
    symbol_info = mt5.symbol_info(symbol) if symbol != "N/A" else None
    digits = symbol_info.digits if symbol_info else 4 # Default to 4 digits for formatting
    
    # --- Local Failure Handling (e.g., failed tick check in place_market_order) ---
    if result is None:
        comment_text = request.get("comment", "Local error: Tick or price info failed.")
        if request.get("volume") == 0.0:
            comment_text = "Trade skipped: Volume calculated as zero."
            
        return {
            "Time": datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'),
            "Status": "SKIPPED/FAILED (Local)",
            "Return Code": "N/A",
            "Comment": comment_text,
            "Symbol": symbol,
            "Type": "BUY" if request.get("type") == mt5.ORDER_TYPE_BUY else "SELL",
            "Volume": request.get("volume", "N/A"),
            "Price": "N/A",
            "SL": "N/A",
            "TP": "N/A",
            "Order Ticket": "N/A",
            "Deal Ticket": "N/A",
            "Position Ticket": "N/A",
        }
        
    # --- MT5 Result Handling ---
    # Implement a robust check for time attributes
    timestamp = time.time() # Default to local time
    if hasattr(result, 'time') and result.time > 0:
        timestamp = result.time
    elif hasattr(result, 'time_done') and result.time_done > 0:
        timestamp = result.time_done
        
    # Create the output dictionary
    output = {
        "Time": datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'),
        "Status": "SUCCESS" if result.retcode == mt5.TRADE_RETCODE_DONE else "FAILURE",
        "Return Code": result.retcode,
        "Comment": result.comment,
        "Symbol": symbol,
        "Type": "BUY" if request.get("type") == mt5.ORDER_TYPE_BUY else "SELL",
        "Volume": request.get("volume"),
        # Format prices using the symbol's digits
        "Price": f"{result.price:.{digits}f}",
        "SL": f"{request.get('sl'):.{digits}f}",
        "TP": f"{request.get('tp'):.{digits}f}",
        "Order Ticket": result.order,
        "Deal Ticket": result.deal,
        "Position Ticket": result.position if hasattr(result, 'position') else "N/A",
    }
    
    return output


# ==============================================================================
# --- MAIN EXECUTION LOGIC ---
# ==============================================================================

def main():
    """Executes the trade bot logic."""
    if connect_mt5():
        
        print("\nStarting multi-symbol trade execution...")
        
        for symbol_name in SYMBOL_LIST:
            # 1. Get Symbol Information
            symbol_info, pip_value = get_symbol_info(symbol_name)
            
            if symbol_info and pip_value:
                # --- PLACE A BUY ORDER ---
                # Note: The 'volume' parameter is omitted here as it's calculated dynamically inside the function.
                trade_result, trade_request = place_market_order(
                    symbol_info, 
                    pip_value, 
                    mt5.ORDER_TYPE_BUY
                )
                
                # 2. Collect the result
                # We check for trade_request to ensure an attempt was made, even if it failed locally
                if trade_request:
                    ALL_TRADE_RESULTS.append(format_result(trade_result, trade_request))
            
            # Add a small delay between trades to respect broker limits
            time.sleep(0.5)

        # 3. Display all results in a single, aggregated DataFrame
        if ALL_TRADE_RESULTS:
            final_df = pd.DataFrame(ALL_TRADE_RESULTS)
            print("\n" + "="*100)
            print("--- AGGREGATED MULTI-SYMBOL TRADE EXECUTION RESULTS (BUY Orders) ---")
            print("="*100)
            # Use max_rows=None to ensure all records are displayed
            pd.set_option('display.max_rows', None) 
            print(final_df.to_string(index=False))
            
        # 4. Shutdown MT5 connection
        mt5.shutdown()
        print("\nMT5 connection successfully shut down.")


if __name__ == "__main__":
    main()

Successfully connected to MT5 terminal.
Account: 40866995

Starting multi-symbol trade execution...

--- Symbol Ready: Volatility 10 Index ---
Digits: 3 | Point: 0.001 | Bid: 5440.191 | Ask: 5440.342
  [ATR Calcs] ATR(14) value: 1.04207143
  [SL Setup] Using ATR-based SL distance: 2.084
  [Risk Calcs] Equity: 6711.26, Risk %: 1.0%, Max Loss $: 67.11, Final Volume: 32.20
  Sending BUY order for Volatility 10 Index with 32.20 lots at 5440.342...

--- Symbol Ready: Volatility 25 Index ---
Digits: 3 | Point: 0.001 | Bid: 2402.513 | Ask: 2402.682
  [ATR Calcs] ATR(14) value: 1.28935714
  [SL Setup] Using ATR-based SL distance: 2.579
  [Risk Calcs] Equity: 6711.26, Risk %: 1.0%, Max Loss $: 67.11, Final Volume: 26.03
  Sending BUY order for Volatility 25 Index with 26.03 lots at 2402.682...

--- Symbol Ready: Volatility 50 Index ---
Digits: 4 | Point: 0.0001 | Bid: 141.7689 | Ask: 141.7879
  [ATR Calcs] ATR(14) value: 0.18322857
  [SL Setup] Using ATR-based SL distance: 0.3665
  [Risk Calcs]

In [34]:
# Cell 8 OLD

def place_market_order(symbol, direction, lot, sl_price=None, tp_price=None):
    mt5_connect()
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print("❌ Market order cannot get tick for", symbol)
        return {"retcode": None, "comment": "NO_TICK"}
    price = float(tick.ask if direction=="BUY" else tick.bid)
    if DRY_RUN:
        # simulated
        return {"retcode":10009, "comment":"DRY_RUN", "price": price}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(lot),
        "type": mt5.ORDER_TYPE_BUY if direction=="BUY" else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": float(sl_price) if sl_price is not None else 0.0,
        "tp": float(tp_price) if tp_price is not None else 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "ppo_multiasset_live",
        "type_filling": mt5.ORDER_FILLING_FOK,
    }
    res = mt5.order_send(req)
    return res

def close_position_by_ticket(ticket):
    pos_list = mt5.positions_get(ticket=ticket)
    if not pos_list:
        return None
    p = pos_list[0]
    symbol = p.symbol
    if getattr(p,"type",None) == 0:
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    else:
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    if DRY_RUN:
        return {"retcode":10009, "comment":"DRY_RUN_CLOSE", "ticket": ticket}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(p.volume),
        "type": order_type,
        "position": int(ticket),
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": "auto_close",
    }
    return mt5.order_send(req)


In [31]:
#NEW
def run_once_predict_and_manage(symbols_list):
    mt5_connect()
    acct = mt5.account_info()
    balance = float(acct.balance) if acct is not None else 10000.0

    header = not os.path.exists(LOG_FILE)

    for mt5_symbol in symbols_list:
        print(f"\n--- {mt5_symbol} ---")

        # Build observation
        obs_tuple = build_obs_for_symbol(mt5_symbol, WINDOW)
        if obs_tuple is None or obs_tuple[0] is None:
            print("No observation for", mt5_symbol)
            continue

        obs, vol, last_price = obs_tuple

        # Predict
        try:
            action, _ = model.predict(obs[np.newaxis, ...], deterministic=True)
            a = int(action[0]) if isinstance(action, (list, tuple, np.ndarray)) else int(action)
        except Exception as e:
            print("Prediction error:", e)
            continue

        print("Signal:", a, "(0=HOLD,1=BUY,2=SELL)")

        # Check existing positions
        positions = get_positions_for_symbol(mt5_symbol)
        print("Existing positions:", len(positions))

        #################################################
        # Auto-close opposing positions
        #################################################
        if a == 1:  # BUY signal => close SELL
            for p in positions:
                if getattr(p, "type", None) == 1:
                    print("Closing opposing SELL ticket", p.ticket)
                    close_position_by_ticket(p.ticket)

        elif a == 2:  # SELL signal => close BUY
            for p in positions:
                if getattr(p, "type", None) == 0:
                    print("Closing opposing BUY ticket", p.ticket)
                    close_position_by_ticket(p.ticket)

        # Refresh position list
        positions = get_positions_for_symbol(mt5_symbol)

        #################################################
        # Trade Execution
        #################################################
        if len(positions) >= MAX_POS_PER_SYMBOL:
            print("Max positions reached for", mt5_symbol)

        else:
            if a == 0:
                print("HOLD")
            else:
                direction = "BUY" if a == 1 else "SELL"
                lot = DEFAULT_LOT

                # SL estimation
                sl_pips = estimate_sl_pips(mt5_symbol, last_price, vol)
                tp_pips = int(sl_pips * TP_MULT)

                sl_price, tp_price = compute_sl_tp_prices(
                    mt5_symbol, last_price, direction, sl_pips, tp_pips
                )

                # Execute trade
                res = place_market_order(mt5_symbol, direction, lot, sl_price, tp_price)

                if isinstance(res, dict):
                    retcode = res.get("retcode")
                    comment = res.get("comment")
                    exec_price = res.get("price", last_price)
                else:
                    retcode = getattr(res, "retcode", None)
                    comment = getattr(res, "comment", "")
                    exec_price = last_price

                # Log trade
                entry = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "symbol": mt5_symbol,
                    "action": direction,
                    "lot": lot,
                    "exec_price": exec_price,
                    "sl_price": sl_price,
                    "tp_price": tp_price,
                    "sl_pips": sl_pips,
                    "tp_pips": tp_pips,
                    "retcode": retcode,
                    "comment": comment,
                    "dry_run": DRY_RUN
                }
                pd.DataFrame([entry]).to_csv(
                    LOG_FILE, mode="a", index=False, header=header
                )
                header = False
                print("Placed", direction, "lot", lot, "retcode", retcode)

        #################################################
        # Trailing Stop Engine
        #################################################
        for p in get_positions_for_symbol(mt5_symbol):

            trail_pips = max(5, int(estimate_sl_pips(mt5_symbol, last_price, vol) // 2))
            pip = pip_value(mt5_symbol)

            if getattr(p, "type", None) == 0:  # BUY
                cur_price = mt5.symbol_info_tick(mt5_symbol).bid
                new_sl = float(cur_price - trail_pips * pip)
            else:  # SELL
                cur_price = mt5.symbol_info_tick(mt5_symbol).ask
                new_sl = float(cur_price + trail_pips * pip)

            if DRY_RUN:
                print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
            else:
                req = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": mt5_symbol,
                    "position": int(p.ticket),
                    "sl": new_sl,
                    "tp": float(p.tp) if getattr(p, "tp", None) else 0.0
                }
                r = mt5.order_send(req)
                print("Modify SL result:", getattr(r, "retcode", None))

    mt5_shutdown()
    print("\nSingle pass complete.")


In [24]:
# Calculate Optimum Lot-size
def calculate_optimal_volume(symbol: str, entry_price: float, sl_price: float, risk_percent: float) -> float:
    """
    Calculates the optimal lot size based on account equity, risk percentage,
    and the stop loss distance, constrained by symbol limits.

    The formula is: OptimalLotSize = (Equity * RiskPercentage) / (abs(EntryPrice - SLPrice) * ContractSize)
    """
    
    # 1. Get Account Equity
    account_info = mt5.account_info()
    if account_info is None:
        print("Error: Could not retrieve account information.")
        return 0.0
    acc_equity = account_info.balance
    
    # 2. Get Symbol Information for contract size and volume limits
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Error: Could not retrieve symbol info for {symbol}.")
        return 0.0
    
    contract_size = symbol_info.trade_contract_size
    min_volume = symbol_info.volume_min
    max_volume = symbol_info.volume_max
    volume_step = symbol_info.volume_step
    
    # Check for zero values to prevent division by zero
    if contract_size == 0:
        print(f"Error: Contract size is zero for {symbol}. Returning minimum volume.")
        return min_volume
        
    # 3. Calculate Maximum Dollar Loss
    max_loss_dollar = acc_equity * risk_percent
    
    # 4. Calculate Loss per Lot (Loss in price difference * Contract Size)
    price_loss = abs(entry_price - sl_price)
    
    # If SL is too close or equal to entry, use min volume as a fallback.
    if price_loss == 0.0:
        print(f"Warning: SL is equal to entry price for {symbol}. Using minimum volume.")
        return min_volume

    loss_per_lot = price_loss * contract_size
    
    # 5. Calculate Optimal Lot Size (Theoretical)
    optimal_lot_size = max_loss_dollar / loss_per_lot
    
    # 6. Apply Volume Step and Min/Max constraints
    
    # Snap the calculated lot size to the nearest allowed volume step (e.g., 0.01)
    optimal_lot_size = round(optimal_lot_size / volume_step) * volume_step
    
    # Apply min/max volume limits
    final_volume = max(min_volume, min(max_volume, optimal_lot_size))
        
    print(f"  [Risk Calcs] Equity: {acc_equity:.2f}, Risk %: {risk_percent*100}%, Max Loss $: {max_loss_dollar:.2f}, Final Volume: {final_volume:.2f}")
    
    # Ensure the final volume is rounded to the step size's precision
    final_volume = round(final_volume, len(str(volume_step).split('.')[-1]) if '.' in str(volume_step) else 0)

    return final_volume


In [53]:
def get_symbol_info(symbol: str): #BEST
    """
    Retrieves and prepares the symbol for trading.
    Returns the symbol information object and the price tick value.
    """
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"--- {symbol} NOT FOUND ---")
        return None, None

    if not symbol_info.visible:
        # Check if the symbol can be added to Market Watch
        if not mt5.symbol_select(symbol, True):
            print(f"--- {symbol} NOT TRADABLE: Not visible and could not be added ---")
            return None, None

    # Get the tick size (point value) for SL/TP calculations
    pip_value = symbol_info.point
    
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"--- {symbol} NOT TRADABLE: Could not get tick info ---")
        return None, None
        
    print(f"\n--- Symbol Ready: {symbol} ---")
    print(f"Digits: {symbol_info.digits} | Point: {symbol_info.point} | Bid: {tick.bid} | Ask: {tick.ask}")
    
    return symbol_info, pip_value


In [57]:
 # This one works well to place trades and calculates optimum the lot-size #BEST
# --- Place trades ---
def place_market_order(symbol_info, pip_value: float, order_type: int):
    """
    Constructs and sends a market order (BUY or SELL) using a dynamically calculated volume.
    
    :param symbol_info: mt5.symbol_info object
    :param pip_value: The size of a single pip (point) for the symbol
    :param order_type: mt5.ORDER_TYPE_BUY or mt5.ORDER_TYPE_SELL
    """
    symbol = symbol_info.name
    
    # Get current price
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Failed to get current tick for {symbol}.")
        # Return a dictionary indicating local failure before calling format_result
        return None, {"symbol": symbol, "type": order_type, "volume": 0.0} 

    # Determine execution price, trade type, and SL/TP prices
    if order_type == mt5.ORDER_TYPE_BUY:
        price = tick.ask
        type_text = "BUY"
        # SL/TP calculation (SL below, TP above entry price for BUY)
        sl_price = price - (SL_OFFSET_PIPS * pip_value)
        tp_price = price + (TP_OFFSET_PIPS * pip_value)
    elif order_type == mt5.ORDER_TYPE_SELL:
        price = tick.bid
        type_text = "SELL"
        # SL/TP calculation (SL above, TP below entry price for SELL)
        sl_price = price + (SL_OFFSET_PIPS * pip_value)
        tp_price = price - (TP_OFFSET_PIPS * pip_value)
    else:
        print("Invalid order type.")
        return None, None
        
    # --- DYNAMIC VOLUME CALCULATION ---
    volume_to_use = calculate_optimal_volume(
        symbol, 
        price, 
        sl_price, 
        RISK_PERCENTAGE
    )
    
    if volume_to_use <= 0.0:
        print(f"Skipping trade for {symbol}: Calculated volume is zero or negative.")
        # Return the request dictionary with 0 volume to be logged as a skipped trade
        return None, {"symbol": symbol, "type": order_type, "volume": 0.0}

    # Prepare the request dictionary
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume_to_use, # <-- Dynamic volume used here
        "type": order_type,
        "price": price,
        # Round SL/TP based on the symbol's number of digits
        "sl": round(sl_price, symbol_info.digits), 
        "tp": round(tp_price, symbol_info.digits), 
        "deviation": DEVIATION,
        "magic": MAGIC,
        "comment": COMMENT,
        "type_time": mt5.ORDER_TIME_GTC, 
        "type_filling": mt5.ORDER_FILLING_FOK, 
    }

    # Send order to MT5
    print(f"  Sending {type_text} order for {symbol} with {volume_to_use:.2f} lots at {price:.{symbol_info.digits}f}...")
    result = mt5.order_send(request)

    return result, request






In [60]:
def format_result(result, request): #BEST
    """
    Formats the order result and request into a clean dictionary.
    Handles both successful and failed execution attempts.
    """
    symbol = request.get("symbol", "N/A")
    symbol_info = mt5.symbol_info(symbol) if symbol != "N/A" else None
    digits = symbol_info.digits if symbol_info else 4 # Default to 4 digits for formatting
    
    # --- Local Failure Handling (e.g., failed tick check in place_market_order) ---
    if result is None:
        comment_text = request.get("comment", "Local error: Tick or price info failed.")
        if request.get("volume") == 0.0:
            comment_text = "Trade skipped: Volume calculated as zero."
            
        return {
            "Time": datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'),
            "Status": "SKIPPED/FAILED (Local)",
            "Return Code": "N/A",
            "Comment": comment_text,
            "Symbol": symbol,
            "Type": "BUY" if request.get("type") == mt5.ORDER_TYPE_BUY else "SELL",
            "Volume": request.get("volume", "N/A"),
            "Price": "N/A",
            "SL": "N/A",
            "TP": "N/A",
            "Order Ticket": "N/A",
            "Deal Ticket": "N/A",
            "Position Ticket": "N/A",
        }
        
    # --- MT5 Result Handling ---
    # Implement a robust check for time attributes
    timestamp = time.time() # Default to local time
    if hasattr(result, 'time') and result.time > 0:
        timestamp = result.time
    elif hasattr(result, 'time_done') and result.time_done > 0:
        timestamp = result.time_done
        
    # Create the output dictionary
    output = {
        "Time": datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'),
        "Status": "SUCCESS" if result.retcode == mt5.TRADE_RETCODE_DONE else "FAILURE",
        "Return Code": result.retcode,
        "Comment": result.comment,
        "Symbol": symbol,
        "Type": "BUY" if request.get("type") == mt5.ORDER_TYPE_BUY else "SELL",
        "Volume": request.get("volume"),
        # Format prices using the symbol's digits
        "Price": f"{result.price:.{digits}f}",
        "SL": f"{request.get('sl'):.{digits}f}",
        "TP": f"{request.get('tp'):.{digits}f}",
        "Order Ticket": result.order,
        "Deal Ticket": result.deal,
        "Position Ticket": result.position if hasattr(result, 'position') else "N/A",
    }
    
    return output


In [61]:
# ==============================================================================
# --- MAIN EXECUTION LOGIC --- BEST
# ==============================================================================

def main():
    """Executes the trade bot logic."""
    if mt5_connect():
        
        print("\nStarting multi-symbol trade execution...")
        
        #for symbol_name in SYMBOL_LIST:
        for symbol_name in SYMBOLS:
            # 1. Get Symbol Information
            symbol_info, pip_value = get_symbol_info(symbol_name)
            
            if symbol_info and pip_value:
                # --- PLACE A BUY ORDER ---
                # Note: The 'volume' parameter is omitted here as it's calculated dynamically inside the function.
                trade_result, trade_request = place_market_order(
                    symbol_info, 
                    pip_value, 
                    mt5.ORDER_TYPE_BUY
                
                )
                
                # 2. Collect the result
                # We check for trade_request to ensure an attempt was made, even if it failed locally
                if trade_request:
                    ALL_TRADE_RESULTS.append(format_result(trade_result, trade_request))
            
            # Add a small delay between trades to respect broker limits
            time.sleep(0.5)

        # 3. Display all results in a single, aggregated DataFrame
        if ALL_TRADE_RESULTS:
            final_df = pd.DataFrame(ALL_TRADE_RESULTS)
            print("\n" + "="*100)
            print("--- AGGREGATED MULTI-SYMBOL TRADE EXECUTION RESULTS (BUY Orders) ---")
            print("="*100)
            # Use max_rows=None to ensure all records are displayed
            pd.set_option('display.max_rows', None) 
            print(final_df.to_string(index=False))
            
        # 4. Shutdown MT5 connection
        mt5.shutdown()
        print("\nMT5 connection successfully shut down.")


if __name__ == "__main__":
    main()

MT5 connected. Version: (500, 5430, '14 Nov 2025')

Starting multi-symbol trade execution...

--- Symbol Ready: Volatility 10 Index ---
Digits: 3 | Point: 0.001 | Bid: 5442.38 | Ask: 5442.531
  [Risk Calcs] Equity: 5989.00, Risk %: 2.0%, Max Loss $: 119.78, Final Volume: 11.98
  Sending BUY order for Volatility 10 Index with 11.98 lots at 5442.531...

--- Symbol Ready: Volatility 25 Index ---
Digits: 3 | Point: 0.001 | Bid: 2402.411 | Ask: 2402.58
  [Risk Calcs] Equity: 5989.00, Risk %: 2.0%, Max Loss $: 119.78, Final Volume: 11.98
  Sending BUY order for Volatility 25 Index with 11.98 lots at 2402.580...

--- Symbol Ready: Volatility 50 Index ---
Digits: 4 | Point: 0.0001 | Bid: 139.8393 | Ask: 139.8583
  [Risk Calcs] Equity: 5989.00, Risk %: 2.0%, Max Loss $: 119.78, Final Volume: 119.78
  Sending BUY order for Volatility 50 Index with 119.78 lots at 139.8583...

--- Symbol Ready: Volatility 75 Index ---
Digits: 2 | Point: 0.01 | Bid: 45281.53 | Ask: 45293.61
  [Risk Calcs] Equity: 5